### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from pandas import json_normalize

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()
NYT_API_KEY = os.getenv("NYT_API_KEY")
TMDB_API_KEY = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [5]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={NYT_API_KEY}"

In [6]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    query_url_page= f"{query_url}&page={str(page)}"
    
    # Make a "GET" request and retrieve the JSON
    articles = requests.get(query_url_page).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for article in articles['response']['docs']:
            reviews_list.append(article)
        # Print the page that was just retrieved
        print(f"Checked page {page}")

        # Print the page number that had no results then break from the loop
    except:
        print(f"No results found on page {page}")
        break

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [54]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [8]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = json_normalize(reviews_list)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [9]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

# def extract_title(s):
#     f = False #Flag to check whether to append or not
#     s1 = "" #Empty string literal to get the title
#     for i,c in enumerate(s):
#         try:
#             if c == '\u2018':
#                 f = True # Start appending
#             elif c == '\u2019' and i != len(s)-1 and  s[i+1] == " ":
#                 f = False #Stop appending
#             elif f and i != len(s)-1:
#                 s1 += c #Append
#         except:
#             print('---------------------------------------------')
#             print(f"Error thrown at: {s}")
#             print('---------------------------------------------')
#             pass
    
#     if len(s1) == 0: #Empty String returns the input string
#         s1 = s
        
#     if s1[-1] == ',': #Incase the headline contains invalid end with ","
#         s1 = s1[:-1]
        
#     return s1

# reviews_df['title'] = reviews_df['headline.main'].apply(extract_title)
# reviews_df

reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",Non

In [10]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody L

In [11]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = reviews_df["title"].to_list()
titles_list

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [13]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + TMDB_API_KEY

In [68]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles_list:
    
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print(f"Sleeping at {request_counter} requests")
        time.sleep(1)
        
    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    res = requests.get(url + title + tmdb_key_string).json()


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
            
        # Get movie id
        movie_id = res["results"][0]["id"]

        # Make a request for a the full movie details
        details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={TMDB_API_KEY}"

        # Execute "GET" request with url
        movie_details = requests.get(details_url).json()
        
        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_details['genres']]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [language['english_name'] for language in movie_details['spoken_languages']]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie_details['production_countries']]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_data = {
            'title': movie_details['title'],
            'original_title': movie_details.get('original_title'),
            'budget': movie_details.get('budget'),
            'genres': genres,
            'language': movie_details.get('original_language'),
            'spoken_languages': spoken_languages,
            'homepage': movie_details.get('homepage'),
            'overview': movie_details.get('overview'),
            'popularity': movie_details.get('popularity'),
            'runtime': movie_details.get('runtime'),
            'revenue': movie_details.get('revenue'),
            'release_date': movie_details.get('release_date'),
            'vote_average': movie_details.get('vote_average'),
            'vote_count': movie_details.get('vote_count'),
            'production_countries': production_countries
        }
        tmdb_movies_list.append(movie_data)

        
        # Print out the title that was found
        print(f"Found {movie_details['title']}")
    except:
        print(f"{title} not found.")

Found The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo not found.
Found You Can Live Forever
Found A Tourist's Guide to Love
Found Other People's Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, A Love Story
Found In from the Side
Found After Love
Found Alcarràs
Found Nelly Knows Mysteries: A Fatal Engagement
Found Lady Chatterley's Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found 2099: The Soldier Protocol
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Mektoub, My Love
Found Stay Prayed Up
Found Benediction
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found After L

In [69]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "language": "es",
        "spoken_languages": [
            "Spanish"
        ],
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.294,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "production_countries": [
            "Argentina"
        ]
    },
   

In [16]:
# Convert the results to a DataFrame
movie_df = pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [18]:
# Merge the New York Times reviews and TMDB DataFrames on title
df = movie_df.merge(reviews_df,
                 on='title')
df

,title,genres,spoken_languages,production_countries,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,"[Drama, Romance]","[English, French]","[Canada, United States of America]",https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]",https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,[Documentary],[English],[United States of America],https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,"[Drama, Crime]","[English, Spanish, Portuguese]",[United States of America],https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,In Search of Fellini,"[Drama, Adventure]",[English],[],https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,subject: Movies;creative_works: In Search of F...,2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,None,None,None,By Monica Castillo,"[{'firstname': 'Monica', 'middlename': None, '...",None
120,The Last Face,[Drama],"[English, Portuguese]",[United States of America],https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
121,Lost in Paris,[Comedy],[French],"[Belgium, France]",https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
122,T

In [19]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_fix = ['genres', 'spoken_languages', 'production_countries', 'byline.person']

# Create a list of characters to remove
characters_remove = ['[',']',"'"]

# Loop through the list of columns to fix
for column in columns_fix:
    # Convert the column to type 'str'
    df[column] = df[column].astype(str)

    # Loop through characters to remove
    for character in characters_remove:
        df[column] = df[column].str.replace(character, '', regex=False)

# Display the fixed DataFrame
df

,title,genres,spoken_languages,production_countries,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"{firstname: Jeannette, middlename: None, lastn...",None
1,You Can Live Forever,"Drama, Romance","English, French","Canada, United States of America",https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"{firstname: Elisabeth, middlename: None, lastn...",None
2,One True Loves,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America",https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"{firstname: Brandon, middlename: None, lastnam...",None
3,The Lost Weekend: A Love Story,Documentary,English,United States of America,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"{firstname: Glenn, middlename: None, lastname:...",None
4,A Thousand and One,"Drama, Crime","English, Spanish, Portuguese",United States of America,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"{firstname: Manohla, middlename: None, lastnam...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,In Search of Fellini,"Drama, Adventure",English,,https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,subject: Movies;creative_works: In Search of F...,2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,None,None,None,By Monica Castillo,"{firstname: Monica, middlename: None, lastname...",None
120,The Last Face,Drama,"English, Portuguese",United States of America,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"{firstname: Glenn, middlename: None, lastname:...",None
121,Lost in Paris,Comedy,French,"Belgium, France",https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"{firstname: Ben, middlename: None, lastname: K...",None
122,The Other Half,"Drama, Romance",English,Canada,ht

In [20]:
# Drop "byline.person" column
df = df.drop('byline.person', axis=1)
df

,title,genres,spoken_languages,production_countries,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,"Drama, Romance","English, French","Canada, United States of America",https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America",https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,Documentary,English,United States of America,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,"Drama, Crime","English, Spanish, Portuguese",United States of America,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,In Search of Fellini,"Drama, Adventure",English,,https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,subject: Movies;creative_works: In Search of F...,2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,None,None,None,By Monica Castillo,None
120,The Last Face,Drama,"English, Portuguese",United States of America,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,None
121,Lost in Paris,Comedy,French,"Belgium, France",https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,None
122,The Other Half,"Drama, Romance",English,Canada,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None


In [21]:
# Delete duplicate rows and reset index
df  = df.drop_duplicates().reset_index(drop=True)
df

,title,genres,spoken_languages,production_countries,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,"Drama, Romance","English, French","Canada, United States of America",https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America",https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,Documentary,English,United States of America,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,"Drama, Crime","English, Spanish, Portuguese",United States of America,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,In Search of Fellini,"Drama, Adventure",English,,https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,subject: Movies;creative_works: In Search of F...,2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,None,None,None,By Monica Castillo,None
120,The Last Face,Drama,"English, Portuguese",United States of America,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,None
121,Lost in Paris,Comedy,French,"Belgium, France",https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,None
122,The Other Half,"Drama, Romance",English,Canada,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None


In [22]:
# Export data to CSV without the index
df.to_csv('filename.csv', index=False)